In [1]:
from kaggle_secrets import UserSecretsClient
wandb_key_label = "WANDB_KEY"
wandb_key= UserSecretsClient().get_secret(wandb_key_label)

In [50]:
import git
git.Repo.clone_from('https://github.com/Lumin-Lab/BerCuration', '/kaggle/working/scarf')

<git.repo.base.Repo '/kaggle/working/scarf/.git'>

In [19]:
!pip install -r /kaggle/working/scarf/requirements.txt

  Using cached matplotlib-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached matplotlib-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.4
    Uninstalling matplotlib-3.8.4:
      Successfully uninstalled matplotlib-3.8.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires matplotlib<3.9,>=3.2, but you have matplotlib 3.9.0 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


In [26]:
import pandas
df = pd.read_csv("/kaggle/input/ber-sample/BER_stratified_sample.csv")

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, 
                               random_state=42,
                               stratify=df[['EnergyRating']])

/tmp/ipykernel_34/467251636.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/ber-sample/BER_stratified_sample.csv")


In [25]:
output_dir = "/kaggle/working/output"
config_dir="/kaggle/working/scarf/configs"
train_data_path = f"{output_dir}/train.csv"
test_data_path = f"{output_dir}/test.csv"
outlier_removed_train_data_path = f"{output_dir}/no_outlier_train.csv"
outlier_removed_test_data_path = f"{output_dir}/no_outlier_test.csv"
scarf_model_name = "scarf"
mlp_model_name = "mlp"
train_embedding_name = "train"
test_embedding_name = "test"

In [ ]:
train['GroundFloorArea_sq_m_'] = train["GroundFloorArea(sq m)"]
test['GroundFloorArea_sq_m_'] = test["GroundFloorArea(sq m)"]

In [34]:
train.to_csv(train_data_path, index=False)
test.to_csv(test_data_path, index=False)

**Train the SCARF Encoder on the Small Train Dataset**

In [36]:
import os
"""The trained scarf model is saved in 
/kaggle/working/output/scarf.pt if you run the following command:
"""

command = f"""
python /kaggle/working/scarf/run_scarf.py \
  --config_dir={config_dir} \
  --output_dir={output_dir} \
  --train_data_path={train_data_path}\
  --batch_size=32 \
  --epochs=1 \
  --lr=3e-5 \
  --emb_dim=32 \
  --encoder_depth=3 \
  --model_name={scarf_model_name} \
  --corruption_rate=0.3 \
  --wandb_project_name='SCARF_Project' \
  --wandb_entity='urbancomp' \
  --wandb_key='{wandb_key}'
"""

os.system(command)

wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240605_162014-6bsbjy3n
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SCARF_Project
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf/runs/6bsbjy3n


Epoch [1/1] - Train Loss: 2.188
Model saved at /kaggle/working/output/scarf.pt


wandb: \ 0.002 MB of 0.019 MB uploaded
wandb: Run history:
wandb: encoder_train/train_loss ██▆▆▅▄▄▃▄▄▅▄▅▃▂▅▃▅▄▃▃▄▃▃▄▄▃▃▃▄▃▃▂▂▂▃▂▁▃▁
wandb: 
wandb: Run summary:
wandb: encoder_train/train_loss 3.53319
wandb: 
wandb: 🚀 View run SCARF_Project at: https://wandb.ai/urbancomp/Scarf/runs/6bsbjy3n
wandb: ⭐️ View project at: https://wandb.ai/urbancomp/Scarf
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240605_162014-6bsbjy3n/logs


0

**Obtain the SCARF embeddings for the Small Train dataset, and save the result**

In [37]:
"""The generated embeddings are saved as a NumPy array in 
/kaggle/working/output/train.npy if you run the following command:
"""
import os
command = f"""
python /kaggle/working/scarf/get_scarf_embedding.py \
  --config_dir={config_dir} \
  --output_dir={output_dir} \
  --data_path={train_data_path} \
  --batch_size=32 \
  --epochs=1 \
  --lr=3e-5 \
  --emb_dim=32 \
  --encoder_depth=3 \
  --model_name={scarf_model_name} \
  --corruption_rate=0.3 \
  --embedding_save_name="train"
"""

os.system(command)

Model loaded from /kaggle/working/output/scarf.pt


0

**Obtain the SCARF embeddings for the Small Test dataset, and save the result**

In [38]:
"""The generated embeddings are saved as a NumPy array in 
/kaggle/working/output/test.npy if you run the following command:
"""
command = f"""
python /kaggle/working/scarf/get_scarf_embedding.py \
  --config_dir={config_dir} \
  --output_dir={output_dir} \
  --data_path={test_data_path} \
  --batch_size=32 \
  --epochs=1 \
  --lr=3e-5 \
  --emb_dim=32 \
  --encoder_depth=3 \
  --model_name={scarf_model_name} \
  --corruption_rate=0.3 \
  --embedding_save_name="train"
"""

os.system(command)

Model loaded from /kaggle/working/output/scarf.pt
Model loaded from /kaggle/working/output/scarf.pt


0

**Load the training and testing embeddings**

In [39]:
from cleanlab import Datalab
import numpy as np
train_emb = np.load(f"{output_dir}/{train_embedding_name}.npy")
test_emb = np.load(f"{output_dir}/{test_embedding_name}.npy")

In [40]:
from cleanlab.outlier import OutOfDistribution
ood = OutOfDistribution()
ood_train_feature_scores = ood.fit_score(features=train_emb)
ood_train_feature_scores = ood.score(features=train_emb)
ood_test_feature_scores = ood.score(features=test_emb)

Fitting OOD estimator based on provided features ...


**Obtain the indices of outliers in both the training and test datasets with cleanlab**

In [10]:
from cleanlab.rank import find_top_issues
top = 100
top_train_ood_features_idxs = find_top_issues(quality_scores=ood_train_feature_scores, 
                                              top=top)
top_test_ood_features_idxs = find_top_issues(quality_scores=ood_test_feature_scores, 
                                              top=top)

In [41]:
train_data_path

'/kaggle/working/output/train.csv'

**Remove the outliers from both the training and test datasets, and save the results**

In [42]:
import pandas as pd
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)
train_df_new = train_df.drop(top_train_ood_features_idxs, inplace=False)
test_df_new = test_df.drop(top_test_ood_features_idxs, inplace=False)

/tmp/ipykernel_34/2134907574.py:2: DtypeWarning: Columns (145,156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_data_path)
/tmp/ipykernel_34/2134907574.py:3: DtypeWarning: Columns (162) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(test_data_path)


In [44]:
import os 
# if not os.path.exists("/kaggle/working/output/"):
#     os.mkdir("/kaggle/working/output/")
train_df_new.to_csv(outlier_removed_train_data_path, index=False)
test_df_new.to_csv(outlier_removed_test_data_path, index=False)

**Train the MLP classifier on the datasets, following the removal of outliers**

In [51]:
"""The trained mlp classifier will be save in /kaggle/working/output/mlp.pt 
if you run the following command:
"""
import os
command = f"""
python /kaggle/working/scarf/run_mlp.py \
  --config_dir={config_dir} \
  --output_dir={output_dir}\
  --train_data_path={outlier_removed_train_data_path} \
  --test_data_path={outlier_removed_test_data_path} \
  --batch_size 32 \
  --epochs 1 \
  --lr 0.00003 \
  --model_name={mlp_model_name} \
  --wandb_project_name "test" \
  --wandb_entity "urbancomp" \
  --wandb_key {wandb_key} \
  --hidden_layer 256 128 64 32 16 \
  --dropout 0.1
"""

os.system(command)

wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240605_163621-kevch2mz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run test
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf-MLP
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf-MLP/runs/kevch2mz


Model saved at /kaggle/working/output/mlp.pt
Epoch [1/1] - Train Loss: 1.350, Train Acc: 0.600, Train F1: 0.444, Test Loss: 1.711, Test Acc: 0.349, Test F1: 0.208
Test Accuracy: 0.349, Test F1: 0.208


wandb: \ 0.114 MB of 0.134 MB uploaded
wandb: Run history:
wandb:      test/A1_acc ▁
wandb:      test/A2_acc ▁
wandb:      test/A3_acc ▁
wandb:      test/B1_acc ▁
wandb:      test/B2_acc ▁
wandb:      test/B3_acc ▁
wandb:      test/C1_acc ▁
wandb:      test/C2_acc ▁
wandb:      test/C3_acc ▁
wandb:      test/D1_acc ▁
wandb:      test/D2_acc ▁
wandb:      test/E1_acc ▁
wandb:      test/E2_acc ▁
wandb:       test/F_acc ▁
wandb:       test/G_acc ▁
wandb:    test/test_acc ▁
wandb:     test/test_f1 ▁
wandb:   test/test_loss ▁
wandb:         train/f1 ▁▁▁▁▁▁▂▃▂▂▂▃▄▂▅▅█▆▃▅▅▄▃▇▄▄▆▅▃▆▇▅▆▇▄▆▄▆▆█
wandb:  train/train_acc ▂▄▂▁▂▃▂▅▃▃▂▄▅▂▆▅▆▆▄▄▅▄▃▆▅▄▆▅▃▆▆▅▆▇▄▇▄▇▆█
wandb: train/train_loss ███████▇▇▆▆▆▅▆▄▄▃▄▅▃▁▆▅▂▃▃▂▂▅▂▁▃▃▂▄▃▃▁▁▁
wandb: 
wandb: Run summary:
wandb:      test/A1_acc 0.0
wandb:      test/A2_acc 0.96881
wandb:      test/A3_acc 0.1964
wandb:      test/B1_acc 0.0
wandb:      test/B2_acc 0.0
wandb:      test/B3_acc 0.06188
wandb:      test/C1_acc 0.71779
wandb:      test/C2_acc 0.48316
wandb: 

0